In [ ]:
%matplotlib ipympl

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pykonal
import seispy

HOME         = os.environ['HOME']
GOOGLE_DRIVE = os.environ['GOOGLE_DRIVE']

In [ ]:
ak135 = pd.read_csv(
    os.path.join(HOME, 'src/pykonal/pykonal/data/AK135F_AVG.csv'),
    header=None,
    names=('depth', 'density', 'vp', 'vs', 'Q_kappa', 'Q_mu')
)
ak135['radius'] = 6371 - ak135['depth']
ak135 = ak135.sort_values('radius')

iasp91 = pd.read_csv(
    os.path.join(HOME, 'src/pykonal/pykonal/data/IASP91.csv'),
    header=None,
    names=('depth', 'radius', 'vp', 'vs')
)

mit2008 = pd.read_csv(
    os.path.join(HOME, 'src/pykonal/pykonal/data/mitp2008.xzv'),
    header=None,
    names=('degrees', 'radius', 'dv'),
    delim_whitespace=True
)
mit2008['radians'] = np.pi - np.radians(mit2008['degrees'] + 16.5)
mit2008 = mit2008.groupby(['radians', 'radius']).mean().reset_index()

In [ ]:
def decorate(func, default=np.nan):
    def wrapper(*args):
        try:
            return (func(*args))
        except Exception:
            return (default)
    return (wrapper)

def rotation_matrix(*args):
    """
    Return the rotation matrix used to rotate a set of cartesian
    coordinates by alpha radians about the z-axis, then beta radians
    about the y'-axis and then gamma radians about the z''-axis.
    """
    if len(args) == 1:
        alpha, beta, gamma = args[0], 0, 0
    elif len(args) == 3:
        alpha, beta, gamma = args
    else:
        raise(ValueError("number of positional arguments must be 1 or 3"))
    aa = np.array([[np.cos(alpha), -np.sin(alpha), 0],
                        [np.sin(alpha), np.cos(alpha), 0],
                        [0, 0, 1]])
    bb = np.array([[np.cos(beta), 0, np.sin(beta)],
                        [0, 1, 0],
                        [-np.sin(beta), 0, np.cos(beta)]])
    cc = np.array([[np.cos(gamma), -np.sin(gamma), 0],
                        [np.sin(gamma), np.cos(gamma), 0],
                        [0, 0, 1]])
    return (aa.dot(bb).dot(cc))


def map_coords(old, old_cs, new_cs, old_origin, rotate=False):
    '''
    Return the coordinates of old in the new reference frame.
    
    :param pykonal.GridND old: Coordinate grid to transform.
    :param str old_cs: Coordinate system to transform from.
    :param str new_cs: Coordinate system to transform to.
    :param old_origin: Coordinates of the origin of old w.r.t. the new frame of reference.
    '''
    old_origin = np.array(old_origin)
    if old_cs.lower() == 'spherical' and new_cs.lower() == 'spherical':
        xx_old = old[...,0] * np.sin(old[...,1]) * np.cos(old[...,2])
        yy_old = old[...,0] * np.sin(old[...,1]) * np.sin(old[...,2])
        zz_old = old[...,0] * np.cos(old[...,1])
        old_origin_xyz = [
            old_origin[0] * np.sin(old_origin[1]) * np.cos(old_origin[2]),
            old_origin[0] * np.sin(old_origin[1]) * np.sin(old_origin[2]),
            old_origin[0] * np.cos(old_origin[1])
        ]
        xx_old_inf  = xx_old + old_origin_xyz[0]
        yy_old_inf  = yy_old + old_origin_xyz[1]
        zz_old_inf  = zz_old + old_origin_xyz[2]
        xyz_old_inf = np.moveaxis(np.stack([xx_old_inf,yy_old_inf,zz_old_inf]), 0, -1)
        
        rr_old_inf  = np.sqrt(np.sum(np.square(xyz_old_inf), axis=-1))
        old         = np.seterr(divide='ignore', invalid='ignore')
        tt_old_inf  = np.arccos(xyz_old_inf[...,2] / rr_old_inf)
        np.seterr(**old)
        pp_old_inf  = np.arctan2(xyz_old_inf[...,1], xyz_old_inf[...,0])
        rtp_old_inf = np.moveaxis(np.stack([rr_old_inf, tt_old_inf, pp_old_inf]), 0, -1)
        return (rtp_old_inf)
    elif old_cs.lower() == 'cartesian' and new_cs.lower() == 'spherical':
        old_origin_xyz = [
            old_origin[0] * np.sin(old_origin[1]) * np.cos(old_origin[2]),
            old_origin[0] * np.sin(old_origin[1]) * np.sin(old_origin[2]),
            old_origin[0] * np.cos(old_origin[1])
        ]
        if rotate is True:
            xyz_old = old[...].dot(rotation_matrix(np.pi/2-old_origin[2], 0, np.pi/2-old_origin[1]))
        else:
            xyz_old = old[...]
        xyz_old    += old_origin_xyz
        xyz_old     = xyz_old
        rr_old      = np.sqrt(np.sum(np.square(xyz_old), axis=-1))
        old         = np.seterr(divide='ignore', invalid='ignore')
        tt_old      = np.arccos(xyz_old[...,2] / rr_old)
        np.seterr(**old)
        pp_old      = np.arctan2(xyz_old[...,1], xyz_old[...,0])
        rr_old_inf  = rr_old
        tt_old_inf  = tt_old
        pp_old_inf  = pp_old
        rtp_old_inf = np.moveaxis(np.stack([rr_old_inf,tt_old_inf, pp_old_inf]), 0, -1)
        return (rtp_old_inf)
    elif old_cs.lower() == 'spherical' and new_cs.lower() == 'cartesian':
        old_origin_xyz = old_origin
        xx_old         = old[...,0] * np.sin(old[...,1]) * np.cos(old[...,2])
        yy_old         = old[...,0] * np.sin(old[...,1]) * np.sin(old[...,2])
        zz_old         = old[...,0] * np.cos(old[...,1])
        xx_old_inf     = xx_old + old_origin_xyz[0]
        yy_old_inf     = yy_old + old_origin_xyz[1]
        zz_old_inf     = zz_old + old_origin_xyz[2]
        xyz_old_inf    = np.moveaxis(np.stack([xx_old_inf,yy_old_inf,zz_old_inf]), 0, -1)
        return (xyz_old_inf)
    elif old_cs.lower() == 'cartesian' and new_cs.lower() == 'cartesian':
        return (old[...] + old_origin)
    else:
        raise (NotImplementedError())
        
def transfer_velocity(old, new, old_origin, rotate=False):
    '''
    Transfer the velocity model from old EikonalSolver to new EikonalSolver
    :param pykonal.EikonalSolver old: The old EikonalSolver to transfer from.
    :param pykonal.EikonalSolver new: The new EikonalSolver to transfer to.
    :param tuple old_origin: The coordinates of the origin of old w.r.t. to the new frame of reference.
    '''
    
    if old.mode not in ('cartesian', 'spherical') or new.mode not in ('cartesian', 'spherical'):
        raise (ValueError('Both EikonalSolvers must be in eithe "cartesian" or "spherical" mode.'))
    vgrid_new_iof = map_coords(new.vgrid[...], new.mode, old.mode, old_origin, rotate=rotate)
    if old.mode == 'spherical' and old.vgrid.min_coords[2] >= 0:
        vgrid_new_iof[...,2] = np.mod(vgrid_new_iof[...,2], 2*np.pi)
    vvi = decorate(
        pykonal.LinearInterpolator3D(old.vgrid, old.vv),
        default=0
    )
    new.vv = np.apply_along_axis(vvi, -1, vgrid_new_iof)
    
def transfer_travel_times(old, new, old_origin, rotate=False, set_alive=False):
    '''
    Transfer the velocity model from old EikonalSolver to new EikonalSolver
    :param pykonal.EikonalSolver old: The old EikonalSolver to transfer from.
    :param pykonal.EikonalSolver new: The new EikonalSolver to transfer to.
    :param tuple old_origin: The coordinates of the origin of old w.r.t. to the new frame of reference.
    '''
    
    if old.mode not in ('cartesian', 'spherical') or new.mode not in ('cartesian', 'spherical'):
        raise (ValueError('Both EikonalSolvers must be in eithe "cartesian" or "spherical" mode.'))
        
    pgrid_new_iof = map_coords(new.pgrid[...], new.mode, old.mode, old_origin, rotate=rotate)
    if old.mode == 'spherical' and old.pgrid.min_coords[2] >= 0:
        pgrid_new_iof[...,2] = np.mod(pgrid_new_iof[...,2], 2*np.pi)
    uui = decorate(pykonal.LinearInterpolator3D(old.pgrid, old.uu))
    
    shape = pgrid_new_iof.shape
    for i1 in range(shape[0]):
        for i2 in range(shape[1]):
            for i3 in range(shape[2]):
                idx = (i1, i2, i3)
                u = uui(pgrid_new_iof[idx])
                if not np.isnan(u):
                    new.uu[idx]       = u
                    new.is_far[idx]   = False
                    new.is_alive[idx] = set_alive
                    new.close.append(idx)

## Global slice


In [ ]:
ak135 = pd.read_csv(
    os.path.join(HOME, 'src/pykonal/pykonal/data/AK135F_AVG.csv'),
    header=None,
    names=('depth', 'density', 'vp', 'vs', 'Q_kappa', 'Q_mu')
)
ak135['radius'] = 6371 - ak135['depth']
ak135 = ak135.sort_values('radius')

iasp91 = pd.read_csv(
    os.path.join(HOME, 'src/pykonal/pykonal/data/IASP91.csv'),
    header=None,
    names=('depth', 'radius', 'vp', 'vs')
)

mit2008 = pd.read_csv(
    os.path.join(HOME, 'src/pykonal/pykonal/data/mitp2008.xzv'),
    header=None,
    names=('degrees', 'radius', 'dv'),
    delim_whitespace=True
)
mit2008['radians'] = np.pi - np.radians(mit2008['degrees'] + 16.5)
mit2008 = mit2008.groupby(['radians', 'radius']).mean().reset_index()

############################
# Initialize the system
int_field                      = pykonal.EikonalSolver()
int_field.mode                 = 'spherical'
int_field.vgrid.min_coords     = mit2008['radius'].min(), np.pi/2, mit2008['radians'].min()
int_field.vgrid.node_intervals = 45.18, -1, 0.00156968
int_field.vgrid.npts           = 64, 1, 1637
int_field.vv                   = np.zeros(int_field.vgrid.npts)
int_field.dv                   = np.zeros(int_field.vgrid.npts)

dvi                            = pykonal.LinearInterpolator3D(
    int_field.vgrid,
    mit2008.sort_values(['radius', 'radians'])['dv'].values.reshape(int_field.vgrid.npts)
)
for ir in range(int_field.vgrid.npts[0]):
    v0 = np.interp(int_field.vgrid[ir,0,0,0], ak135['radius'], ak135['vp'])
    for it in range(int_field.vgrid.npts[1]):
        for ip in range(int_field.vgrid.npts[2]):
            dv = dvi(int_field.vgrid[ir, it, ip])
            int_field.dv[ir, it, ip] = dv
            int_field.vv[ir, it, ip] = v0 * (1 + dv*1e-2)

near_field_origin               = (6303.25, np.pi/2, 2.45401555)
near_field                      = pykonal.EikonalSolver()
near_field.mode                 = 'spherical'
near_field.vgrid.min_coords     = 1, np.pi/2, 0
near_field.vgrid.node_intervals = 5, 1, np.pi/40
near_field.vgrid.npts           = 110, 1, 81
transfer_velocity(int_field, near_field, near_field_origin)

far_field_origin               = (6348, np.pi/2, 5 * np.pi/16)
far_field                      = pykonal.EikonalSolver()
far_field.mode                 = 'cartesian'
far_field.vgrid.min_coords     = -250, -250, 0
far_field.vgrid.node_intervals = 50, 10, 1
far_field.vgrid.npts           = 11, 26, 1
far_field.vv                   = np.zeros(far_field.vgrid.npts)
transfer_velocity(int_field, far_field, far_field_origin, rotate=True)
############################

############################
# SOLVE the system
for it in range(near_field.pgrid.npts[1]):
    for ip in range(near_field.pgrid.npts[2]):
        idx = (0, it, ip)
        near_field.uu[idx] = near_field.pgrid[idx + (0,)] / near_field.vv[idx]
        near_field.close.append(idx)
        near_field.is_far[idx] = False
near_field.solve()

pgrid = transfer_travel_times(
    near_field,
    int_field,
    (-near_field_origin[0], near_field_origin[1], near_field_origin[2]),
    set_alive=True
)
int_field.solve()

transfer_travel_times(int_field, far_field, far_field_origin, rotate=True, set_alive=False)
############################

############################
# Plot the system
plt.close('all')
fig = plt.figure(figsize=(8,6))
ax1 = fig.add_subplot(2, 1, 1, aspect=1)
ax2 = fig.add_subplot(2, 1, 2, aspect=1)


it = 0

qmesh = ax1.pcolormesh(
    int_field.pgrid[:,it,:,0] * np.sin(int_field.pgrid[:,it,:,1]) * np.cos(int_field.pgrid[:,it,:,2]),
    int_field.pgrid[:,it,:,0] * np.sin(int_field.pgrid[:,it,:,1]) * np.sin(int_field.pgrid[:,it,:,2]),
    int_field.dv[:,it,:],
    cmap=plt.get_cmap('seismic_r'),
    shading='gouraud',
    vmin=-1,
    vmax=1
)
cbar = fig.colorbar(qmesh, ax=ax1)
cbar.set_label(r'$\frac{dv}{v_P}$ [\%]')

qmesh = ax2.pcolormesh(
    int_field.pgrid[:,it,:,0] * np.sin(int_field.pgrid[:,it,:,1]) * np.cos(int_field.pgrid[:,it,:,2]),
    int_field.pgrid[:,it,:,0] * np.sin(int_field.pgrid[:,it,:,1]) * np.sin(int_field.pgrid[:,it,:,2]),
    int_field.uu[:,it,:],
    cmap=plt.get_cmap('jet_r'),
    shading='gouraud'
)
cbar = fig.colorbar(qmesh, ax=ax2)
cbar.set_label(r'Travel time [s]')

In [ ]:
src_idx = (62, 0, 1380)
############################
# Initialize the system
int_field2                      = pykonal.EikonalSolver()
int_field2.mode                 = 'spherical'
int_field2.vgrid.min_coords     = mit2008['radius'].min(), np.pi/2, mit2008['radians'].min()
int_field2.vgrid.node_intervals = 45.18, -1, 0.00156968
int_field2.vgrid.npts           = 64, 1, 1637
int_field2.vv                   = np.zeros(int_field2.vgrid.npts)
int_field2.dv                   = np.zeros(int_field2.vgrid.npts)

dvi                            = pykonal.LinearInterpolator3D(
    int_field2.vgrid,
    mit2008.sort_values(['radius', 'radians'])['dv'].values.reshape(int_field2.vgrid.npts)
)
for ir in range(int_field2.vgrid.npts[0]):
    v0 = np.interp(int_field2.vgrid[ir,0,0,0], ak135['radius'], ak135['vp'])
    for it in range(int_field2.vgrid.npts[1]):
        for ip in range(int_field.vgrid.npts[2]):
            dv = dvi(int_field2.vgrid[ir, it, ip])
            int_field2.dv[ir, it, ip] = dv
            int_field2.vv[ir, it, ip] = v0 * (1 + dv*1e-2)

int_field2.uu[src_idx]       = 0
int_field2.is_far[src_idx]   = False
int_field2.is_alive[src_idx] = True
int_field2.close.append(src_idx)
int_field2.solve()

############################
# Plot the system
plt.close('all')
fig = plt.figure(figsize=(8,6))
ax1 = fig.add_subplot(2, 1, 1, aspect=1)
ax2 = fig.add_subplot(2, 1, 2, aspect=1)


it = 0

qmesh = ax1.pcolormesh(
    int_field2.pgrid[:,it,:,0] * np.sin(int_field2.pgrid[:,it,:,1]) * np.cos(int_field2.pgrid[:,it,:,2]),
    int_field2.pgrid[:,it,:,0] * np.sin(int_field2.pgrid[:,it,:,1]) * np.sin(int_field2.pgrid[:,it,:,2]),
    int_field2.dv[:,it,:],
    cmap=plt.get_cmap('seismic_r'),
    shading='gouraud',
    vmin=-1,
    vmax=1
)
cbar = fig.colorbar(qmesh, ax=ax1)
cbar.set_label(r'$\frac{dv}{v_P}$ [\%]')

qmesh = ax2.pcolormesh(
    int_field2.pgrid[:,it,:,0] * np.sin(int_field2.pgrid[:,it,:,1]) * np.cos(int_field2.pgrid[:,it,:,2]),
    int_field2.pgrid[:,it,:,0] * np.sin(int_field2.pgrid[:,it,:,1]) * np.sin(int_field2.pgrid[:,it,:,2]),
    int_field2.uu[:,it,:] - int_field.uu[:,it,:],
    cmap=plt.get_cmap('jet_r'),
    vmin=-2,
    vmax=2,
    shading='gouraud'
)
cbar = fig.colorbar(qmesh, ax=ax2)
cbar.set_label(r'Travel time [s]')

In [ ]:
np.argwhere(np.all(int_field.pgrid[...] == near_field_origin, axis=-1))

In [ ]:
int_field.pgrid[...].shape

In [ ]:
int_field.pgrid[62,0,1380,2]

In [ ]:
near_field_origin

# TESTING

In [1]:
%matplotlib ipympl

import matplotlib.pyplot as plt
import numpy as np
import pykonal

In [29]:
far_field                      = pykonal.EikonalSolver()
far_field.vgrid.min_coords     = -25, -25, 0
far_field.vgrid.node_intervals = 5, 5, 1
far_field.vgrid.npts           = 11, 11, 1
far_field.vv                   = np.ones(far_field.vgrid.npts)
for iy in range(far_field.vgrid.npts[1]):
    far_field.vv[:,iy,:] *= iy + 1

src_idx = (5, 5, 0)
far_field.uu[src_idx]     = 0
far_field.is_far[src_idx] = False
far_field.close.append(src_idx)
far_field.solve()

origin                          = far_field.pgrid[src_idx]
near_field                      = pykonal.EikonalSolver()
near_field.vgrid.min_coords     = 0, 0, 0
near_field.vgrid.node_intervals = 1, 1, 1
near_field.vgrid.npts           = 11, 11, 1
near_field.vv                   = np.ones(near_field.vgrid.npts)

near_field.transfer_velocity_from(far_field, origin)
near_field.transfer_travel_times_from(far_field, origin)

plt.close('all')
fig = plt.figure()
ax  = fig.add_subplot(1, 1, 1)
ax.pcolormesh(
    far_field.pgrid[:,:,0,0],
    far_field.pgrid[:,:,0,1],
    far_field.uu[:,:,0],
    shading='gouraud'
)
grid = near_field.vgrid.map_to('cartesian', origin)
ax.pcolormesh(
    grid[:,:,0,0],
    grid[:,:,0,1],
    near_field.uu[:,:,0],
    vmin=far_field.uu.min(),
    vmax=far_field.uu.max(),
    edgecolor='k',
    linewidth=0.01
)

FigureCanvasNbAgg()

## Test Cartesian to Cartesian transformation

In [ ]:
old_origin             = (-5, 3, 0)
x0, y0                 = old_origin[0], old_origin[1]

old_xyz                = pykonal.GridND()
old_xyz.min_coords     = -5, -5, 0
old_xyz.node_intervals = 1, 1, 1
old_xyz.npts           = 11, 11, 1

new_xyz = old_xyz.map_to('cartesian', old_origin)

plt.close('all')
fig = plt.figure(figsize=(8,4))
ax  = fig.add_subplot(1, 2, 1)
ax.scatter(0, 0, marker='o', c='r')
ax.scatter(old_xyz[:,:,0,0], old_xyz[:,:,0,1], s=100, marker='o', edgecolor='k', facecolor='none')
ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)

ax  = fig.add_subplot(1, 2, 2)
ax.scatter(x0, y0, marker='o', c='r')
ax.scatter(0, 0, marker='o', c='k')
ax.scatter(new_xyz[:,:,0,0], new_xyz[:,:,0,1], s=100, marker='o', edgecolor='k', facecolor='none')
ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)

## Test spherical to Cartesian transformation

In [ ]:
old_origin             = (-3, -5, 0)
x0, y0                 = old_origin[0], old_origin[1]

old_rtp                = pykonal.GridND(coord_sys='spherical')
old_rtp.min_coords     = 0, np.pi/2, 0
old_rtp.node_intervals = 1, 1, np.pi/10
old_rtp.npts           = 11, 1, 11
old_xx                 = old_rtp[...,0] * np.sin(old_rtp[...,1]) * np.cos(old_rtp[...,2])
old_yy                 = old_rtp[...,0] * np.sin(old_rtp[...,1]) * np.sin(old_rtp[...,2])

new_xyz = old_rtp.map_to('cartesian', old_origin)

plt.close('all')
fig = plt.figure(figsize=(8,4))
ax  = fig.add_subplot(1, 2, 1)
ax.scatter(0, 0, marker='o', c='r')
ax.scatter(old_xx[:,0,:], old_yy[:,0,:], s=100, marker='o', edgecolor='k', facecolor='none')
ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)

ax  = fig.add_subplot(1, 2, 2)
ax.scatter(x0, y0, marker='o', c='r')
ax.scatter(0, 0, marker='o', c='k')
ax.scatter(new_xyz[:,0,:,0], new_xyz[:,0,:,1], s=100, marker='o', edgecolor='k', facecolor='none')
ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)

## Test cartesian to spherical transformation

In [ ]:
old_origin              = (10, np.pi/2, 7*np.pi/4)

grid_xyz                = pykonal.GridND(coord_sys='cartesian')
grid_xyz.min_coords     = -5, -5, 0
grid_xyz.node_intervals = 1, 1, 1
grid_xyz.npts           = 11, 11, 1

grid_rtp = grid_xyz.map_to('spherical', old_origin)
x0       = old_origin[0] * np.sin(old_origin[1]) * np.cos(old_origin[2])
y0       = old_origin[0] * np.sin(old_origin[1]) * np.sin(old_origin[2])
xx       = grid_rtp[...,0] * np.sin(grid_rtp[...,1]) * np.cos(grid_rtp[...,2])
yy       = grid_rtp[...,0] * np.sin(grid_rtp[...,1]) * np.sin(grid_rtp[...,2])

plt.close('all')
fig = plt.figure(figsize=(8,4))
ax  = fig.add_subplot(1, 2, 1)
ax.scatter(0, 0, marker='o', c='r')
ax.scatter(grid_xyz[:,:,0,0], grid_xyz[:,:,0,1], s=100, marker='o', edgecolor='k', facecolor='none')
ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)

ax  = fig.add_subplot(1, 2, 2)
ax.scatter(x0, y0, marker='o', c='r')
ax.scatter(0, 0, marker='o', c='k')
ax.scatter(xx[:,:,0], yy[:,:,0], s=100, marker='o', edgecolor='k', facecolor='none')
ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)

## Test spherical to spherical transformation

In [ ]:
old_origin              = (5, np.pi/2, 3*np.pi/4)
x0                      = old_origin[0] * np.sin(old_origin[1]) * np.cos(old_origin[2])
y0                      = old_origin[0] * np.sin(old_origin[1]) * np.sin(old_origin[2])

old_rtp                = pykonal.GridND(coord_sys='spherical')
old_rtp.min_coords     = 0, np.pi/2, 0
old_rtp.node_intervals = 1, 1, np.pi/10
old_rtp.npts           = 11, 1, 11
old_xx                 = old_rtp[...,0] * np.sin(old_rtp[...,1]) * np.cos(old_rtp[...,2])
old_yy                 = old_rtp[...,0] * np.sin(old_rtp[...,1]) * np.sin(old_rtp[...,2])

new_rtp = old_rtp.map_to('spherical', old_origin)
new_xx                 = new_rtp[...,0] * np.sin(new_rtp[...,1]) * np.cos(new_rtp[...,2])
new_yy                 = new_rtp[...,0] * np.sin(new_rtp[...,1]) * np.sin(new_rtp[...,2])

plt.close('all')
fig = plt.figure(figsize=(8,4))
ax  = fig.add_subplot(1, 2, 1)
ax.scatter(0, 0, marker='o', c='r')
ax.scatter(old_xx[:,0,:], old_yy[:,0,:], s=100, marker='o', edgecolor='k', facecolor='none')
ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)

ax  = fig.add_subplot(1, 2, 2)
ax.scatter(x0, y0, marker='o', c='r')
ax.scatter(0, 0, marker='o', c='k')
ax.scatter(new_xx[:,0,:], new_yy[:,0,:], s=100, marker='o', edgecolor='k', facecolor='none')
ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)